In [1]:
import json
from lxml import etree

In [2]:
tree = etree.parse('all_young_dudes.html', etree.HTMLParser())
root = tree.getroot()

### Book Title

In [3]:
book_data = {}

In [4]:
# title
book_title = root.xpath('head/title/text()')[0]
book_data['title'] = book_title
book_title

'All the Young Dudes - MsKingBean89 - Harry Potter - J. K. Rowling'

### Tags

I'm leaving these for now. It's formatted much better on the AO3 page. Quite crap to work with in the html download

In [5]:
# tags
tags = root.xpath("//div[@class='meta']//dl[@class='tags']")[0]

In [6]:
# TODO: handle tags and read hrefs
tag_labels = tags.xpath(".//dt//text()")
tag_values = tags.xpath(".//dd")

assert len(tag_labels) == len(tag_values)


In [7]:
# tag_data = []
# for label, values in zip(tag_labels, tag_values):

#     # clean up label
#     label = label[:-1] if label.endswith(':') else label
#     label = label.lower()
#     label = label.replace(' ', '_')

#     # will get stats from the main page
#     if label == 'stats':
#         continue

#     # convert values to list
#     # most already are but some (like language) aren't
#     if not isinstance(values, list):
#         values = [values]

#     values_list = []
#     for value in values:

#         attrs = {}

#         # most tag values are links so I can grab both the href and the link text
#         if value.tag == 'a':
#             attrs['text'] = value.text
#             attrs['link'] = value.get('href')

#         # some tag values are just text
#         if value.tag == 'dd':
#             attrs['text'] = value.text.strip()
#             attrs['link'] = None

#     tag_data.append({
#         'name': label,
#         'values': values_list
#     })

# tag_data

# test tags contain basic labels (language, stats)

# book_data['tags'] = tag_data

### Chapters

In [8]:
# create list of dicts for every chapter
book = root.xpath("//div[@id='chapters']")[0]

chapter_metadata = book.xpath(".//div[@class='meta group']") # //*[@class='heading']//text()")
chapter_content = book.xpath(".//div[@class='userstuff']")

# should have the same amount of chapter titles and content
assert len(chapter_metadata) == len(chapter_content)

chapters_data = []
for meta, content in zip(chapter_metadata, chapter_content):

    title = meta.xpath("./*[@class='heading']//text()")
    title = '\n'.join(title).strip()

    # everything in the meta group that isn't part of the chapter title
    summary_and_notes_raw = meta.xpath(".//text()[not(ancestor::*[@class='heading'])]")
    summary_and_notes_stripped = [item.strip() for item in summary_and_notes_raw]
    summary_and_notes = '\n'.join(summary_and_notes_stripped).strip()

    text_raw = content.xpath(".//text()")
    text_stripped = [item.strip() for item in text_raw]
    text = '\n'.join(text_stripped).strip()

    chapter_dict = {
        'title': title,
        'summary_and_notes': summary_and_notes,
        'content': text
    }

    chapters_data.append(chapter_dict)

book_data['chapters'] = chapters_data

print(f'{len(chapters_data)} chapters found')

188 chapters found


In [10]:
# write data to json
with open('book_data.json', 'w') as f:
    json.dump(book_data, f)


### Tests

In [27]:
assert book_data['title'] != None

In [28]:
assert len(book_data['chapters']) > 0

In [29]:
for chapter in book_data['chapters']:

    assert chapter['title'] is not None
    assert chapter['title'].strip() != ''

    assert chapter['content'] is not None
    assert chapter['content'].strip() != ''
